In [45]:
import requests
from requests import get
from requests.exceptions import RequestException
from user_agent import generate_user_agent, generate_navigator
from contextlib import closing
from bs4 import BeautifulSoup
from lxml import html
import re
import csv
import pandas as pd
import os
from pathlib import Path

headers = generate_navigator()
print(headers)
import urllib3

urllib3.disable_warnings()

{'os_id': 'win', 'navigator_id': 'firefox', 'platform': 'Windows NT 5.1; WOW64', 'oscpu': 'Windows NT 5.1; WOW64', 'build_version': '48.0', 'build_id': '20160908035026', 'app_version': '5.0 (Windows)', 'app_name': 'Netscape', 'app_code_name': 'Mozilla', 'product': 'Gecko', 'product_sub': '20100101', 'vendor': '', 'vendor_sub': '', 'user_agent': 'Mozilla/5.0 (Windows NT 5.1; WOW64; rv:48.0) Gecko/20100101 Firefox/48.0'}


In [46]:
headers = {'User-Agent': 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Safari/65.1.15'}    

In [59]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True, timeout=1, headers=headers, verify=False)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except Exception as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [48]:
def get_all_links(soup, homepage):
    pages = []
    for item in soup.find_all('a'):
        page = item.get('href')
        if page != None:
            if page[:3] == '../':
                page = homepage + page[3:]
        pages.append(page)
    return list(set(pages))

In [54]:
def get_url_info(url):
    try:
        homepage = 'https://' + url
        raw_html = simple_get(homepage)
    except:
        try:
            homepage = 'http://' + url
            raw_html = simple_get(homepage)
        except requests.exceptions.ConnectionError:
            r.status_code = "Connection refused"
    
    if raw_html != None:
        soup = BeautifulSoup(raw_html, 'html.parser')
    else:
        return None
    
    try:
        title = soup.title.get_text()
    except AttributeError:
        title = None
        
    if '|' in title:  # TODO is this the case for all?
        title = title.split('|')[0]
    text = soup.get_text()
    
    pages = get_all_links(soup, homepage)
    
    return pages, text, title

In [55]:
pages, text, title = get_url_info("www.baeckerei-grobe.de/")
print(title)
print(pages)

Bäckerei Grobe 
['https://www.baeckerei-grobe.de/pages/impressum.php', 'https://www.baeckerei-grobe.de/pages/aktuelles.php?id=143', 'https://www.baeckerei-grobe.de/pages/grobe.php?id=3', 'https://www.innungsbaecker.de/baeckerfinder/baecker/c2d3a2cd-915c-4f93-89ef-ae33c91d1e04/', 'https://www.umwelt.nrw.de/landwirtschaft/ernaehrungswirtschaft/meisterwerknrw-ehrenpreis-des-landes-nrw/', 'https://www.baeckerei-grobe.de/pages/aktuelles.php?id=113', 'https://www.baeckerei-grobe.de/pages/grobe.php?id=11', 'https://www.baeckerei-grobe.de/pages/grobe.php?id=4', 'https://www.baeckerei-grobe.de/pages/grobe.php', 'https://www.baeckerei-grobe.de/pages/grobe.php?id=5', 'https://www.baeckerei-grobe.de/pages/grobe.php?id=2', 'https://www.baeckerei-grobe.de/pages/aktuelles.php?id=142', 'https://www.baeckerei-grobe.de/pages/aktuelles.php', 'https://www.baeckerei-grobe.de/pages/produkte.php', 'https://www.facebook.com/Baeckermeister-Grobe-GmbH-CoKG-231016537300128/', 'https://www.baeckerei-grobe.de/page

In [56]:
pages, text, title = get_name("www.mvv-edl.de")
print(title)
print(pages)

None
2018 MVV Energie AG


In [8]:
def get_company_urls():
    company_urls = []
    
    data = pd.read_csv('20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', encoding='latin1', names=['Domain', 'Section', 'Code'])
    
    for index, row in data.iterrows():
        company_urls.append(row['Domain'])
        
    return company_urls

In [9]:
def check_string_in_list(string, arr):
    arr = list(set(arr))
    
    for item in arr:
        if string in item:
            return item

In [10]:
def get_company_sites(str_list, pages):
    sites = []
    
    for item in str_list:
        # Problem iterating 3 times TODO
        site = check_string_in_list(item, pages)
        sites.append(site)
    
    return sites

# leander: scraping

In [11]:
all_df = pd.read_csv('../data/raw/20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', encoding='latin1')

In [12]:
dom = all_df['Domain']

In [13]:
def get_c_title(text):
    end_c = re.search("©", text).end()
    start_nl = re.search("\n", text[end_c:]).start()
    title = text[end_c+1:end_c+start_nl]
    
    return title

def get_name(url):
        try:
            pages, text, title = get_url_info(url)
#             pages, title = get_url_info(url)
            try:
                c_title = get_c_title(text)
            except AttributeError:
                c_title = "PAGE TITLE: " + title
            specific_pages = get_company_sites(['impressum'], pages)
#             infos[url] = [title, specific_pages, None] # TODO 
            return [c_title, specific_pages, None]
        except TypeError:
            return [None, None, None]

In [14]:
get_url_info("www.wahler.de")

Error during requests to https://www.wahler.de : HTTPSConnectionPool(host='www.wahler.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16cb2c2748>: Failed to establish a new connection: [Errno 111] Connection refused',))


In [15]:
data = pd.DataFrame(all_df["Domain"])
import numpy as np
datas = np.array_split(data, 10)
datas[0].shape

(1078, 1)

In [16]:
add = re.compile("(\d){5} [a-zA-Z]*")
full = re.compile("[\w-]+ (\d)[\w]* (\d){5} [a-zA-Z]*")
street = re.compile("[\w-]+ (\d)")
def get_address(soupy):
    xs = soupy.find_all(text=re.compile("(\d){5} [a-zA-Z]*"));
    if len(xs) == 0:
        return none
    else:
        x = xs[0]
        t = x.strip()
        f_match = full.match(t)
        if f_match:
            return f_match.group(1)
        else:
            return x.previous_sibling.previous_sibling.strip() + " " + t

In [17]:
add = re.compile("(\d){5} [a-zA-Z]*")
full = re.compile("[\w-]+ (\d)[\w]* (\d){5} [a-zA-Z]*")
street = re.compile("[\w-]+ (\d)")
def get_address(soupy):
    xs = soupy.find_all(text=re.compile("(\d){5} [a-zA-Z]*"));
    if len(xs) == 0:
        return none
    else:
        x = xs[0]
        t = x.strip()
        f_match = full.match(t)
        if f_match:
            return f_match.group(1)
        else:
            return x.previous_sibling.previous_sibling.strip() + " " + t

In [18]:
import sys
import urllib
from urllib.request import urlopen
from urllib.request import urlopen, Request
def map_to_addr(url):
    try:
        try:
            content = urlopen(Request("https://" + url, headers=headers), timeout = 10)
        except:
            try:
                content = urlopen(Request("http://" + url, headers=headers), timeout = 10)
            except:
                raise
        soup = BeautifulSoup(content, 'html.parser')
        x = soup.find_all(text=add);
        return get_address(soup)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        return "NO!"
        #raise

In [24]:
def process_url(url):
    info1 = get_name(url)
    specific_pages = info1[1]
    info2 = None
    if (specific_pages is not None):
        first = specific_pages [0]
        if specific_pages[0] is not None:
            subpage = specific_pages[0]
            if not subpage.startswith("/"):
                subpage = "/" + subpage
            info2 = map_to_addr(url + subpage)
    else:
        print("nothing found")
    return (info1, info2)

In [31]:
get_name("www.mvv-edl.de")

Error during requests to https://www.mvv-edl.de : HTTPSConnectionPool(host='www.mvv-edl.de', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.mvv-edl.de' doesn't match either of '*.mvv-energie.de', 'mvv-energie.de'",),))


[None, None, None]

In [60]:
import dask.dataframe as dd
from dask.multiprocessing import get as get_sched
data = pd.DataFrame(dom)

for i in range(0,20):
    ddata = dd.from_pandas(datas[i], npartitions=90)

    res = ddata.map_partitions(lambda df: df.apply((lambda row: process_url(*row)), axis=1)).compute(get=get_sched)  
    
    res.to_csv("impressum_data_" + str(i) + ".csv")

Error during requests to https://foo : HTTPSConnectionPool(host='foo', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16e887fa58>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))
nothing found
Error during requests to https://foo : HTTPSConnectionPool(host='foo', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16e887fb00>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))
nothing found
Error during requests to https://www.biotest.de : HTTPSConnectionPool(host='www.biotest.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16e88902e8>: Failed to establish a new connection: [Errno 111] Connection refused',))
nothing found
nothing found
Unexpected error: 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Unexpected error: <class 'NameError'>
Unexpected error: <class 'urllib.error.HTTPError'>
Unexpected error: <class 'urllib.error.HTTPError'>
Error during requests to https://www.kikxxl.de : HTTPSConnectionPool(host='www.kikxxl.de', port=443): Read timed out. (read timeout=1)
nothing found
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'urllib.error.HTTPError'>
nothing found
Unexpected error: <class 'urllib.error.HTTPError'>
Error during requests to https://www.mietwaesche.de : HTTPSConnectionPool(host='www.mietwaesche.de', port=443): Read timed out. (read timeout=1)
nothing found
Error during requests to https://www.andritz-hydro.de : HTTPSConnectionPool(host='www.andritz-hydro.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16ca3be1d0>: Failed to establish a new connection: [Errno 111] Connection refused',))
nothing found
Unexpected error: <class 'urllib.error.HTTPError'>
Unex

/home/leanderk/.local/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


nothing found
Error during requests to https://www.dorow-online.de : HTTPSConnectionPool(host='www.dorow-online.de', port=443): Read timed out. (read timeout=1)
nothing found
Unexpected error: <class 'urllib.error.HTTPError'>
nothing found
Error during requests to https://www.damstahl.de : HTTPSConnectionPool(host='www.damstahl.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16fa7c57f0>: Failed to establish a new connection: [Errno 111] Connection refused',))
nothing found
Error during requests to https://www.protectionone.de : HTTPSConnectionPool(host='www.protectionone.de', port=443): Read timed out. (read timeout=1)
nothing found
Error during requests to https://www.tertianum-muenchen.de : HTTPSConnectionPool(host='muenchen.tertianum.de', port=443): Read timed out. (read timeout=1)
nothing found
nothing found
nothing found
nothing found
nothing found
Error during requests to https://www.tel-rot

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Unexpected error: <class 'NameError'>
nothing found
Error during requests to https://www.huegler-gmbh.de : HTTPConnectionPool(host='www.huegler-gmbh.de', port=80): Read timed out. (read timeout=1)
nothing found
nothing found
Unexpected error: <class 'ssl.CertificateError'>
Unexpected error: <class 'urllib.error.HTTPError'>
nothing found
nothing found
Error during requests to https://www.maier-kartonagen.de : HTTPSConnectionPool(host='www.ikmaier.com', port=443): Read timed out. (read timeout=1)
nothing found
Unexpected error: <class 'AttributeError'>
Error during requests to https://www.jjweiler.de : HTTPSConnectionPool(host='www.jjweiler.de', port=443): Read timed out. (read timeout=1)
nothing found
Error during requests to https://www.j-scheidig.de : HTTPSConnectionPool(host='www.j-scheidig.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
nothing found
Error during requests to https://www.stoefen.de

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Unexpected error: <class 'NameError'>
Error during requests to https://www.muenzinger.de : HTTPSConnectionPool(host='www.muenzinger.de', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f16ca16bb70>: Failed to establish a new connection: [Errno 111] Connection refused',))
nothing found
Error during requests to https://www.namlifeinkost.de : HTTPSConnectionPool(host='www.namlifeinkost.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_read_bytes', 'tlsv1 alert internal error')],)",),))
nothing found
Error during requests to https://www.n-cut24.de : HTTPSConnectionPool(host='www.n-cut24.de', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_read_bytes', 'tlsv1 alert internal error')],)",),))
nothing found
Error during requests to https://www.noris-blattgold.de : HTTP

IndexError: list index out of range

In [81]:
imp = pd.read_csv("impressum_data_0.csv", header=None)
import ast
imp[1] = imp[1].apply(ast.literal_eval)
imp[["infos", "street"]] = imp[1].apply(pd.Series)
imp.head()

,0,1,infos,street
0,0,"([Hoberg und Driesch GmbH & Co. KG 2018, [impr...","[Hoberg und Driesch GmbH & Co. KG 2018, [impre...",Theodorstraße 101 40472 Düsseldorf
1,1,"([None, None, None], None)","[None, None, None]",None
2,2,"([None, None, None], None)","[None, None, None]",None
3,3,"([None, None, None], None)","[None, None, None]",None
4,4,"([None, None, None], None)","[None, None, None]",None


In [87]:
import requests
def get_lat_long(street):
    if street is None:
        return [-1.0, -1.0]
    params = {"query" : street}
    url = 'https://atlas.microsoft.com/search/address/json?subscription-key=vi0uYbzziKhl9ywM3jqW_wSAJ2uNDuDIDJJg8Hc6IGE&api-version=1.0&query=Theodorstraße 101 40472 Düsseldorf'
    response = requests.get(url, params=params)
    entry = response.json()["results"][0]["entryPoints"][0]["position"]
    lat = entry["lat"]
    long = entry["lon"]
    return [lat, long]

In [91]:
for i in range(0,20):
    print(i)
    imp = pd.read_csv("impressum_data_" + str(i) + ".csv", header=None)
    import ast
    imp[1] = imp[1].apply(ast.literal_eval)
    imp[["infos", "street"]] = imp[1].apply(pd.Series)
    imp.head()
    #ddata = dd.from_pandas(imp["street"], npartitions=90)

    imp["geocoded"] = imp["street"].apply(get_lat_long)
    
    imp.to_csv("impressum_data_geocoded_" + str(i) + ".csv")

0
1
2
3
4
5
6
7
8
9
10


FileNotFoundError: File b'impressum_data_10.csv' does not exist

In [90]:
import dask.dataframe as dd
from dask.multiprocessing import get as get_sched
for i in range(0,20):
    imp = pd.read_csv("impressum_data_" + str(i) + ".csv", header=None)
    import ast
    imp[1] = imp[1].apply(ast.literal_eval)
    imp[["infos", "street"]] = imp[1].apply(pd.Series)
    imp.head()
    ddata = dd.from_pandas(imp["street"], npartitions=90)

    res = ddata.map_partitions(lambda df: df.apply((lambda row: get_lat_long(*row)), axis=1)).compute(get=get_sched)  
    
    res.to_csv("impressum_data_geocoded_" + str(i) + ".csv")

ValueError: Metadata inference failed in `lambda`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("<lambda>() got an unexpected keyword argument 'axis'",)

Traceback:
---------
  File "/home/leanderk/.local/lib/python3.6/site-packages/dask/dataframe/utils.py", line 137, in raise_on_meta_error
    yield
  File "/home/leanderk/.local/lib/python3.6/site-packages/dask/dataframe/core.py", line 3585, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "<ipython-input-90-1e8648f98f66>", line 11, in <lambda>
    res = ddata.map_partitions(lambda df: df.apply((lambda row: get_lat_long(*row)), axis=1)).compute(get=get_sched)
  File "/home/leanderk/.local/lib/python3.6/site-packages/pandas/core/series.py",

In [ ]:
import ast
all_met[1] = all_met[1].apply(ast.literal_eval)
all_met[["abstract", "description", "keywords", "crashed"]] = all_met[1].apply(pd.Series)